In [12]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from PDF_Tool import pdf_tool
from CSV_Tool import csv_tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
import pandas as pd
from itertools import cycle
from json import tool
import folium
from langchain.tools import Tool

In [9]:
# Define the PromptTemplate
prompt = PromptTemplate(
    template="""
    You are an AI agent responsible for choosing the right tool to answer the user's query and dont use interent or online search. Below is the conversation so far:

    {messages}

    Based on the user's latest query, decide which tool to use:
    
    - If the query is related to Bandar, use the `pdf_tool` to extract relevant information from a PDF document.
    - If the query is related to Drones, use the `csv_tool` to extract relevant information from a CSV file.
    
    after using the right tool answer based on information that you got from the tool.
    """,
    input_variables=["messages"]
)

In [11]:
def visualize_geo_points(
    path, category_column, lat_column='lat', lon_column='lon', output_file='map.html'
):
    """
    Visualize geospatial points on a map with category-based pin colors.
 
    Parameters:
        path(str): path to csv file.
        category_column (str): Column name for the categories (e.g., 'type').
        lat_column (str): Column name for latitude values.
        lon_column (str): Column name for longitude values.
        output_file (str): Name of the output HTML file.
 
    Returns:
        folium.Map: Folium map object.
    """
    df=pd.read_csv(path)
    # Predefined color palette
    predefined_colors = [
        'red', 'blue', 'green', 'purple', 'orange', 'darkred',
        'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue'
    ]
 
    # Assign colors dynamically to unique categories
    unique_categories = df[category_column].unique()
    color_cycle = cycle(predefined_colors)  # Cycle through colors if categories > colors
    color_map = {category: next(color_cycle) for category in unique_categories}
 
    # Create a Folium map centered on the mean location
    center_lat = df[lat_column].mean()
    center_lon = df[lon_column].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)
 
    # Add points to the map with dynamic colors
    for _, row in df.iterrows():
        # Generate a dynamic popup text
        popup_content = "<br>".join(
            [f"{col}: {row[col]}" for col in df.columns if col not in [lat_column, lon_column]]
        )
 
        folium.Marker(
            location=[row[lat_column], row[lon_column]],
            popup=popup_content,
            tooltip=row[category_column],
            icon=folium.Icon(color=color_map.get(row[category_column], 'gray'))  # Default to 'gray'
        ).add_to(m)
 
    # Save map to file
    m.save(output_file)
    return m

 

In [13]:
# Create a LangChain Tool
visualize_geo_points_tool = Tool(
    name="visualize_geo_points",
    func=lambda path, category_column, lat_column='lat', lon_column='lon', output_file='map.html': visualize_geo_points(
        path, category_column, lat_column, lon_column, output_file
    ),
    description=(
        "This tool visualizes geospatial data from a CSV file. It takes the path to the CSV file, "
        "the name of the category column (e.g., 'type'), latitude, and longitude column names. It generates "
        "an interactive HTML map and returns the path to the saved HTML file."
    )
)

In [14]:
# Load the LLM model
llm = ChatOllama(model='mistral')

# Define tools
tools = [pdf_tool, csv_tool,visualize_geo_points_tool]

# Connect the LLM with the prompt
llm_prompt = LLMChain(llm=llm, prompt=prompt)

# Define memory to remember the chat history
memory = MemorySaver()

In [4]:
# Create the react agent
agent = create_react_agent(
    tools=tools,
    model=llm,
    state_modifier=prompt,
    checkpointer=memory
)

In [15]:
# Configuration
config = {"configurable": {"thread_id": "1"}}

# User input
user_input = "do visualize_geo_points_tool"
# Prepare the input messages
input_messages = [HumanMessage(user_input)]

# Run the agent
response = agent.invoke({"messages": input_messages}, config=config)

# Print the response
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

 Since the user's latest query is about visualizing geo points, we need to use the `visualize_geo_points_tool`. After using this tool, assuming it returns a set of visualized geo-points, the response would be "Here are your requested visualized geo-points."
